In [1]:
import datetime
import os
import numpy as np
import pandas as pd
import random
import re
import requests
import selenium
import shutil
import time
import tqdm

# Selenium Dependent
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.core.utils import ChromeType


from bs4 import BeautifulSoup
from collections import defaultdict, Counter
from pprint import pprint


In [42]:
def folder_maker(folder2make):
    # create directory to store html files
    if os.path.isdir(folder2make):
        shutil.rmtree(folder2make)
    os.mkdir(folder2make)


def nap_time(min_seconds=40):
    timer = np.random.randint(1,60) + min_seconds
    time.sleep(timer)

In [3]:
path_ = '/Volumes/GoogleDrive/My Drive/_509/projekt_509/coursera_courses.csv'
courses_df = pd.read_csv(path_)
courses_df.sample(3)

,Title,Link
11,Ask questions to make data-driven decisions,https://www.coursera.org/learn/ask-questions-m...
4,OOP Python,https://www.coursera.org/learn/object-oriented...
6,"Python Object Basics: List, Strings, and Files",https://www.coursera.org/learn/python-basic-st...


In [54]:
driver = webdriver.Chrome(ChromeDriverManager(chrome_type=ChromeType.CHROMIUM).install())


/var/folders/7s/yfj6qr7s1l7gp41rtvvtnbp00000gn/T/ipykernel_58224/14171497.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager(chrome_type=ChromeType.CHROMIUM).install())


In [55]:
# 500 pages == 12.5k reviews
max_pages = 500
review_container_class = 'cds-63 review review-text review-page-review m-b-2 css-0 cds-64'


for index, row in courses_df.iterrows():
    link = row['Link']

    course_name = link.split('/')[-2]

    # initialize to starting page
    driver.get(link + '?page=1')

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # dynamic page checker, returns number of pages for course
    len_pages = len(soup.find_all('li', style='display:inline-block')) - 2 # last link is next page, plus 0 initial counting
    total_pages = soup.find_all('li', style='display:inline-block')[len_pages]
    total_pages = int(total_pages.text)
    # 500 pages == 12.5k reviews
    if total_pages > max_pages:
        total_pages = max_pages

    review_items = []
    for page_num in range(1,total_pages):
        nap_time()

        reviews_containers = soup.find_all('div', class_=review_container_class)

        for review_set in reviews_containers:
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            text = review_set.select_one('.reviewText').p.text

            wanted_span = len(review_set.select('span')) - 1
            helpful_count = review_set.select('span')[wanted_span].text

            review_items.append({'text':text,
                                 'helpful_count':helpful_count})

        # click next page
        next_button_num = len(driver.find_elements(By.CLASS_NAME, "_1lutnh9y")) - 1
        driver.find_elements(By.CLASS_NAME, "_1lutnh9y")[next_button_num].click()


    pd.DataFrame()
    nap_time()


In [41]:
# driver.close()

# Step Through FN

In [7]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# dynamic page checker, returns number of pages for course
len_pages = len(soup.find_all('li', style='display:inline-block')) - 2 # last link is next page, plus 0 initial counting
total_pages = soup.find_all('li', style='display:inline-block')[len_pages]
total_pages = int(total_pages.text)

In [8]:
review_container_class = 'cds-63 review review-text review-page-review m-b-2 css-0 cds-64'
reviews_containers = soup.find_all('div', class_=review_container_class)

In [10]:
review_items = []
for review_set in reviews_containers:
    text = review_set.select_one('.reviewText').p.text

    wanted_span = len(review_set.select('span')) - 1
    helpful_count = review_set.select('span')[wanted_span].text

    review_items.append({'text':text,
                         'helpful_count':helpful_count})

# click next page
next_button_num = len(driver.find_elements(By.CLASS_NAME, "_1lutnh9y")) - 1
driver.find_elements(By.CLASS_NAME, "_1lutnh9y")[next_button_num].click()